In [51]:
import numpy as np
import numpy.random as random
import importlib
import tqdm
import time
import keras
import pickle
from IPython.display import display, clear_output

In [2]:
import tttn_funcs as T

2022-11-19 07:05:35.767811: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [67]:
importlib.reload(T);

Simple  5 x 5 game


In [9]:
import os

In [11]:
os.chdir("./final_results/5x5_FCN/d_1/")

In [4]:
start_time = time.perf_counter()
games = [T.simulateGame() for _ in tqdm.tqdm(range(10000))]
print( time.perf_counter() - start_time, "secs")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:24<00:00, 118.26it/s]

84.571753793 secs


In [22]:
start_time = time.perf_counter()
model = T.getModel()
X_train, X_test, y_train, y_test = T.gamesToWinLossData(games)
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100)
# model.save_weights("./model")
model.load_weights("./model").expect_partial()
model.save_weights("./epoch_0/model")
print( time.perf_counter() - start_time, "secs")

2.3911762410000392 secs


In [35]:
epoch = 1

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = T.getModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

In [36]:
start_time = time.perf_counter()
epoch = 2

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = T.getModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")
print(time.perf_counter() - start_time, "Seconds")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:59<00:00,  6.60s/it]


Training new model
664.016752255 Seconds


In [57]:
g = new_t_games[2][1]
for i in range(1, len(g)+1):
    clear_output()
    display(T.printMoves(g[:i]))
    input()

|  X X 0 0 |
|      0 0 |
|X X   0 0 |
|    X 0 X |
|0 X X 0 X |
Winner: 2


None

In [66]:
n_models = 3
TResults = [[0 for e1 in range(n_models)] for e2 in range(n_models)]
for e1 in range(n_models):
    model1 = T.getModel()
    model1.load_weights("./epoch_"+str(e1)+"/model").expect_partial()
    for e2 in range(e1, n_models):
        print("Play between ", e1, " and ", e2,"\t", end="")
        model2 = T.getModel()
        model1.load_weights("./epoch_"+str(e2)+"/model").expect_partial()
        tg, n_faults = T.gen_tournament(p1=model1, p2=model2, n=1, rnd=0, tqdm_disable=True)
        TResults[e1][e2]=tg
        winners = np.array([g[0] for g in tg])
        nDraws = np.sum(winners == 0)
        nW1 = np.sum(winners == 1)
        nW2 = np.sum(winners == 2)
        print("(nD, nW1, nW2)=", (nDraws, nW1, nW2), " n_faults=", n_faults)
        

Play between  0  and  0 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  0  and  1 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  0  and  2 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  1  and  1 	(nD, nW1, nW2)= (0, 0, 1)  n_faults= 0
Play between  1  and  2 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  2  and  2 	(nD, nW1, nW2)= (0, 0, 1)  n_faults= 0


## CNN model

In [222]:
def getCNNModel():
    board = T.initBoard()
    numCells = len(board)*len(board[0])
    num_rows = len(board)
    outcomes = 3
    CNNmodel = keras.models.Sequential()
    CNNmodel.add( keras.layers.InputLayer(input_shape=(numCells, )))
    CNNmodel.add( keras.layers.Reshape( target_shape = (num_rows, num_rows, 1)))
    CNNmodel.add( keras.layers.Conv2D(filters = 32, kernel_size = 3, padding = "same"))
    CNNmodel.add( keras.layers.Reshape( target_shape = (1,5*5*32)))
    CNNmodel.add( keras.layers.BatchNormalization())
    CNNmodel.add( keras.layers.Dense(100, activation = "relu"))
    CNNmodel.add( keras.layers.BatchNormalization())
    CNNmodel.add( keras.layers.Dense(20, activation = "relu"))
    CNNmodel.add( keras.layers.BatchNormalization())
    CNNmodel.add( keras.layers.Dense(3, activation = "relu"))
    CNNmodel.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    CNNmodel.add( keras.layers.Reshape(target_shape=(3,)))
    # training the model
    # CNNmodel.compile(loss = keras.losses.categorical_crossentropy,
    #           optimizer = keras.optimizers.SGD(lr = 0.01),
    #           metrics =['accuracy'])
    CNNmodel.compile(optimizer='adam',
                  loss=keras.losses.MeanSquaredError(),
                  metrics=['accuracy'])
#    [CNNmodel.input_shape, CNNmodel.output_shape]
    return CNNmodel
getCNNModel().summary()

Model: "sequential_93"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_106 (Reshape)       (None, 5, 5, 1)           0         
                                                                 
 conv2d_44 (Conv2D)          (None, 5, 5, 32)          320       
                                                                 
 reshape_107 (Reshape)       (None, 1, 800)            0         
                                                                 
 batch_normalization_90 (Bat  (None, 1, 800)           3200      
 chNormalization)                                                
                                                                 
 dense_291 (Dense)           (None, 1, 100)            80100     
                                                                 
 batch_normalization_91 (Bat  (None, 1, 100)           400       
 chNormalization)                                    

In [208]:
print(os.getcwd())
os.chdir("../../5x5_CNN/d_1")

/Users/luchinsky/Work/DATA7300_Tian/DATA7300_Tian_Final/final_results/5x5_CNN/d_1


In [223]:
start_time = time.perf_counter()
cnnModel = getCNNModel()
X_train, X_test, y_train, y_test = T.gamesToWinLossData(games)
# history = cnnModel.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100)
# cnnModel.save_weights("./model")
cnnModel.load_weights("./model").expect_partial()
cnnModel.save_weights("./epoch_0/model")
print( time.perf_counter() - start_time, "secs")

2.5542090680010006 secs


In [233]:
epoch = 1

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = getCNNModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [12:13<00:00,  7.34s/it]


Training new model


In [234]:
epoch = 2

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = getCNNModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [10:13<00:00,  6.14s/it]


Training new model


In [235]:
epoch = 3

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = getCNNModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [12:38<00:00,  7.59s/it]


Training new model


In [309]:
epoch = 4

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = getCNNModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.9)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [12:07<00:00,  7.27s/it]


Training new model


In [310]:
epoch = 5

dir_name = "./epoch_"+str(epoch)+"/"
#if not os.path.exists(dir_name):
os.makedirs(dir_name)
#
old_model = getCNNModel()
old_model.load_weights("./epoch_"+str(epoch-1)+"/model").expect_partial();
new_model = keras.models.clone_and_build_model(old_model)
print("Simulation")
new_t_games, n_fault = T.gen_tournament(p1=new_model, p2=old_model, n=100, rnd=0.5)
with open(dir_name+"/t_games.dat", "wb") as file:
    pickle.dump(new_t_games, file)
# #

print("Training new model")
X_train, X_test, y_train, y_test = T.gamesToWinLossData( [g[1] for g in new_t_games], train_ratio=0.99 )
new_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=100, verbose = 0)
new_model.save_weights(dir_name+"/model")

Simulation


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [12:47<00:00,  7.67s/it]


Training new model


In [311]:
n_models = 5
TResults = [[0 for e1 in range(n_models)] for e2 in range(n_models)]
for e1 in range(n_models):
    model1 = getCNNModel()
    model1.load_weights("./epoch_"+str(e1)+"/model").expect_partial()
    for e2 in range(e1, n_models):
        print("Play between ", e1, " and ", e2,"\t", end="")
        model2 = getCNNModel()
        model1.load_weights("./epoch_"+str(e2)+"/model").expect_partial()
        tg, n_faults = T.gen_tournament(p1=model1, p2=model2, n=1, rnd=0.1, tqdm_disable=True)
        TResults[e1][e2]=tg
        winners = np.array([g[0] for g in tg])
        nDraws = np.sum(winners == 0)
        nW1 = np.sum(winners == 1)
        nW2 = np.sum(winners == 2)
        print("(nD, nW1, nW2)=", (nDraws, nW1, nW2), " n_faults=", n_faults)
        

Play between  0  and  0 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  0  and  1 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  0  and  2 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  0  and  3 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  0  and  4 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  1  and  1 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  1  and  2 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  1  and  3 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  1  and  4 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  2  and  2 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  2  and  3 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  2  and  4 	(nD, nW1, nW2)= (0, 1, 0)  n_faults= 0
Play between  3  and  3 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  3  and  4 	(nD, nW1, nW2)= (1, 0, 0)  n_faults= 0
Play between  4  and  4 	(nD, nW1, nW2)= (0, 0, 1)  n_faults= 0


In [307]:
def printMoves(moves, markers = dict({0:' ', 1:"X", 2:"\x1b[31m0\x1b[0m", 10:' ', 11:"\x1b[1;30mX\x1b[0m", 12:"\x1b[1;31m0\x1b[0m"})):
    board = T.movesToBoard(moves)
    for i in range(len(board)):
        print("|", end="")
        for j in range(len(board[i])):
            if all([i,j] == moves[-1][1]):
                print(markers[10+board[i][j]], end=" ")
            else:
                print(markers[board[i][j]], end=" ")                
        print("|")
    print("Winner:", T.getWinner(board))


In [313]:
moves = TResults[3][4][0][1]
for i in range(1, len(moves)+1):
    clear_output()
    print("Move", i)
    display(printMoves(moves[:i]))
    input()

Move 25
|X X X X 0 |
|0 X 0 X 0 |
|0 X X X 0 |
|X 0 0 0 0 |
|X X 0 0 X |
Winner: 0


None

In [297]:
moves

[(1, array([0, 3])),
 (2, array([1, 2])),
 (1, array([0, 1])),
 (2, array([1, 0])),
 (1, array([2, 3])),
 (2, array([3, 3])),
 (1, array([1, 1])),
 (2, array([0, 2])),
 (1, array([1, 4])),
 (2, array([4, 3])),
 (1, array([3, 4])),
 (2, array([0, 0])),
 (1, array([4, 4])),
 (2, array([3, 2])),
 (1, array([2, 4])),
 (2, array([2, 2])),
 (1, array([3, 1])),
 (2, array([2, 0])),
 (1, array([2, 1])),
 (2, array([1, 3])),
 (1, array([4, 0])),
 (2, array([4, 2]))]

In [254]:
print('\x1b[1;31m'+'Hello world'+'\x1b[0m')

Hello world


In [256]:
print('\x1b[1;31m'+'Hello world'+'\x1b[0m')

Hello world


In [260]:
print('\x1b[;31m'+'Hello world'+'\x1b[0m')

Hello world


In [265]:
print('\x1b[1;30m'+'Hello world'+'\x1b[0m')

Hello world


In [274]:
def printMoves(moves):
    T.printBoard(T.movesToBoard(moves[:-1]))


In [298]:
printMoves(moves)

|0 X 0 X   |
|0 X 0 0 X |
|0 X 0 X X |
|  X 0 0 X |
|X   0 0 X |
Winner: 2


In [288]:
printMoves(moves[:-1])

|0 X 0 X   |
|0 X 0 0 X |
|0 X 0 X X |
|  X 0 0 X |
|0     0 X |
Winner: -1


In [273]:
[1,2,3][:-1]

[1, 2]

In [299]:
moves[:-1]

[(1, array([0, 3])),
 (2, array([1, 2])),
 (1, array([0, 1])),
 (2, array([1, 0])),
 (1, array([2, 3])),
 (2, array([3, 3])),
 (1, array([1, 1])),
 (2, array([0, 2])),
 (1, array([1, 4])),
 (2, array([4, 3])),
 (1, array([3, 4])),
 (2, array([0, 0])),
 (1, array([4, 4])),
 (2, array([3, 2])),
 (1, array([2, 4])),
 (2, array([2, 2])),
 (1, array([3, 1])),
 (2, array([2, 0])),
 (1, array([2, 1])),
 (2, array([1, 3])),
 (1, array([4, 0]))]